In [1]:
import pandas as pd

In [3]:
monthly = 'MS'
daily = 'D'

In [58]:
from utils.preprocessing import load_daily_data, load_monthly_data

daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

daily_train

,unique_id,ds,y
0,Austria,2015-01-01,22.34
1,Austria,2015-01-02,22.34
2,Austria,2015-01-03,22.34
3,Austria,2015-01-04,22.34
4,Austria,2015-01-05,36.18
...,...,...,...
3648,Austria,2024-12-27,121.79
3649,Austria,2024-12-28,122.12
3650,Austria,2024-12-29,110.49
3651,Austria,2024-12-30,118.13


### daily

In [59]:
# MACHINE LEARNING:
## Machine Learning Models:
from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


models_ml = [
    LinearRegression(),
    HuberRegressor(epsilon=1.35, alpha=1e-3),
    RandomForestRegressor(n_estimators=400, max_depth=20, min_samples_leaf=5, max_features='sqrt', random_state=42),
    LGBMRegressor(objective="regression", boosting_type="gbdt", num_leaves=31, max_depth=-1, learning_rate=0.03, n_estimators=1000, min_child_samples=30, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=0.1, random_state=42)
]

ml_forecast = MLForecast(models=models_ml, freq='D', lags=[356])  ##LAGS? ##DATE FEATURES? ##WINDOWS? 
ml_forecast.fit(daily_train[['unique_id','ds','y']])
fc_val = ml_forecast.predict(h = 356, X_df=daily_val)
df_val_ml = fc_val.merge(daily_val[['unique_id','ds','y']], on=['unique_id','ds'], how='left')
df_val_ml

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 3297, number of used features: 1
[LightGBM] [Info] Start training from score 81.136837


,unique_id,ds,LinearRegression,HuberRegressor,RandomForestRegressor,LGBMRegressor,y
0,Austria,2025-01-01,90.882059,66.129095,121.399521,205.096941,84.86
1,Austria,2025-01-02,89.774236,65.052914,76.019394,77.387660,121.51
2,Austria,2025-01-03,82.394611,57.884068,127.556705,152.038649,117.59
3,Austria,2025-01-04,80.864760,56.397913,103.966356,157.109517,126.19
4,Austria,2025-01-05,85.530512,60.930400,226.141138,215.762592,124.96
...,...,...,...,...,...,...,...
351,Austria,2025-12-18,94.609972,69.750529,79.110159,90.577273,135.16
352,Austria,2025-12-19,94.706686,69.844482,84.681183,90.577273,126.37
353,Austria,2025-12-20,91.298226,66.533374,191.753740,205.096941,111.08
354,Austria,2025-12-21,93.537318,68.708513,166.226126,132.385685,106.41


In [ ]:
def mergeHolidays_daily(df):
    import holidays
    aut_holidays = holidays.Austria(years=range(2015, 2026))
    
    df['is_holiday'] = df['ds'].isin(aut_holidays.keys()).astype('int8')
    return df

def mergeHolidays_monthly(df, df_daily):
    if 'is_holiday' not in df_daily.columns:
        df_daily = mergeHolidays_daily(df_daily)

    daily_helper = df_daily[['unique_id', 'ds', 'is_holiday']].copy()
    daily_helper['year_month'] = daily_helper['ds'].dt.to_period('M')

    monthly_holidays = (
        daily_helper
        .groupby(['unique_id', 'year_month'], as_index=False)['is_holiday']
        .sum()
        .rename(columns={'is_holiday': 'count_holiday'})
    )

    monthly_holidays['ds'] = monthly_holidays['year_month'].dt.to_timestamp()
    monthly_holidays = monthly_holidays.drop(columns=['year_month'])

    df = df.merge(monthly_holidays, on=['unique_id', 'ds'], how='left')
    df['count_holiday'] = df['count_holiday'].fillna(0).astype('int16')

    return df


In [62]:
daily_train_holidays = mergeHolidays_daily(daily_train)
daily_val_holidays = mergeHolidays_daily(daily_val)
# daily_test_holidays = mergeHolidays_daily(daily_test)
    
# display(daily_val_holidays)

monthly_train_holidays = mergeHolidays_monthly(monthly_train, daily_train)
monthly_val_holidays = mergeHolidays_monthly(monthly_val, daily_val)
# monthly_test_holidays = mergeHolidays_monthly(monthly_test, daily_test)

# display(monthly_train_holidays)

C:\Users\loren\AppData\Local\Temp\ipykernel_24664\618623678.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['ds'].isin(aut_holidays.keys()).astype('int8')
C:\Users\loren\AppData\Local\Temp\ipykernel_24664\618623678.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['ds'].isin(aut_holidays.keys()).astype('int8')


,unique_id,ds,y,is_holiday
0,Austria,2025-01-01,84.86,1
1,Austria,2025-01-02,121.51,0
2,Austria,2025-01-03,117.59,0
3,Austria,2025-01-04,126.19,0
4,Austria,2025-01-05,124.96,0
...,...,...,...,...
351,Austria,2025-12-18,135.16,0
352,Austria,2025-12-19,126.37,0
353,Austria,2025-12-20,111.08,0
354,Austria,2025-12-21,106.41,0


,unique_id,ds,y,count_holiday
0,Austria,2015-01-01,29.935161,2
1,Austria,2015-02-01,36.695000,0
2,Austria,2015-03-01,31.297419,0
3,Austria,2015-04-01,29.778333,1
4,Austria,2015-05-01,25.329677,3
...,...,...,...,...
115,Austria,2024-08-01,84.477097,1
116,Austria,2024-09-01,82.400667,0
117,Austria,2024-10-01,85.353548,1
118,Austria,2024-11-01,130.205667,1


In [63]:
## ML with LAGS
from mlforecast.lag_transforms import RollingMean, RollingStd


ml_lags = [1, 7, 28]
data_features = ['dayofweek', 'month', 'quarter']
lag_transforms = {
    # Short-term volatility
    1: [RollingStd(window_size=3)],
    # Weekly trend
    7: [RollingMean(window_size=3)],
    # Monthly smoothing
    28: [RollingMean(window_size=14)],
}
ml_forecast_lags = MLForecast(models=models_ml, freq='D', lags=ml_lags, date_features=data_features)

ml_forecast_lags.fit(df=daily_train, static_features=[])
fc_val = ml_forecast_lags.predict(h = 356, X_df=daily_val)

df_val_ml_lags = fc_val.merge(daily_val[['unique_id','ds','y']], on=['unique_id','ds'], how='left')
df_val_ml_lags

c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\sklearn\linear_model\_huber.py:348: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 792
[LightGBM] [Info] Number of data points in the train set: 3625, number of used features: 7
[LightGBM] [Info] Start training from score 76.709903


,unique_id,ds,LinearRegression,HuberRegressor,RandomForestRegressor,LGBMRegressor,y
0,Austria,2025-01-01,110.754521,112.012111,119.622166,111.027133,84.86
1,Austria,2025-01-02,111.937611,111.296547,124.089237,114.987644,121.51
2,Austria,2025-01-03,110.557038,110.056939,130.982597,112.601700,117.59
3,Austria,2025-01-04,104.687090,105.851838,127.018573,105.706346,126.19
4,Austria,2025-01-05,94.261500,98.403926,120.162366,101.464575,124.96
...,...,...,...,...,...,...,...
351,Austria,2025-12-18,100.092907,61.897894,252.355121,223.231113,135.16
352,Austria,2025-12-19,95.169929,58.914483,235.095295,220.744915,126.37
353,Austria,2025-12-20,85.561983,53.125963,225.891317,205.336663,111.08
354,Austria,2025-12-21,71.287130,44.578546,221.093562,193.839842,106.41


## MONTHLY

In [64]:
ml_forecast = MLForecast(models=models_ml, freq='MS', lags=[12])  ##LAGS? ##DATE FEATURES? ##WINDOWS? 
ml_forecast.fit(monthly_train[['unique_id','ds','y']])
fc_val = ml_forecast.predict(h = 12, X_df=monthly_val)
df_val_ml = fc_val.merge(monthly_val[['unique_id','ds','y']], on=['unique_id','ds'], how='left')
df_val_ml

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 37
[LightGBM] [Info] Number of data points in the train set: 108, number of used features: 1
[LightGBM] [Info] Start training from score 81.233215
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,unique_id,ds,LinearRegression,HuberRegressor,RandomForestRegressor,LGBMRegressor,y
0,Austria,2025-01-01,82.626296,57.187356,204.252605,141.409010,134.178710
1,Austria,2025-02-01,78.269654,52.643626,206.544193,162.194929,140.771071
2,Austria,2025-03-01,77.536276,51.878755,206.621740,162.194929,103.604194
3,Austria,2025-04-01,76.390283,50.683549,202.619044,162.194929,81.323667
4,Austria,2025-05-01,77.912735,52.271380,206.621740,162.194929,71.016774
5,Austria,2025-06-01,79.103390,53.513166,211.233088,162.194929,66.672667
6,Austria,2025-07-01,77.766021,52.118365,206.621740,162.194929,87.980000
7,Austria,2025-08-01,83.739660,58.348532,130.506461,141.409010,74.744194
8,Austria,2025-09-01,83.143830,57.727114,206.242483,141.409010,92.587667
9,Austria,2025-10-01,83.991157,58.610829,130.506461,141.409010,108.226452


## MONTH WITH LAG

In [65]:
## ML with LAGS
from mlforecast.lag_transforms import RollingMean, RollingStd


ml_lags = [1, 3, 12]
data_features = ['month', 'quarter', 'year']
lag_transforms = {
    # Short-term monthly volatility
    1: [RollingStd(window_size=3)],
    # Quarterly trend
    3: [RollingMean(window_size=2)],
    # Yearly smoothing
    12: [RollingMean(window_size=3)],
}

ml_forecast_lags = MLForecast( models=models_ml, freq='MS', lags=ml_lags, date_features=data_features, lag_transforms=lag_transforms)

ml_forecast_lags.fit(df=monthly_train, static_features=[])

fc_val = ml_forecast_lags.predict(h=12, X_df=monthly_val)

df_val_ml_lags = fc_val.merge(
    monthly_val[['unique_id', 'ds', 'y']],
    on=['unique_id', 'ds'],
    how='left'
)

df_val_ml_lags

c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\sklearn\linear_model\_huber.py:348: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000037 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 250
[LightGBM] [Info] Number of data points in the train set: 106, number of used features: 9
[LightGBM] [Info] Start training from score 82.284558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,unique_id,ds,LinearRegression,HuberRegressor,RandomForestRegressor,LGBMRegressor,y
0,Austria,2025-01-01,123.746321,107.804804,133.261543,155.110704,134.178710
1,Austria,2025-02-01,115.618462,103.131897,121.872761,176.434232,140.771071
2,Austria,2025-03-01,124.336797,98.471823,111.040617,176.434232,103.604194
3,Austria,2025-04-01,122.859891,92.607152,106.738656,176.434232,81.323667
4,Austria,2025-05-01,128.071471,88.659191,102.561537,129.980502,71.016774
5,Austria,2025-06-01,135.333040,85.743875,98.028306,196.823206,66.672667
6,Austria,2025-07-01,136.134020,83.732848,100.632357,232.520310,87.980000
7,Austria,2025-08-01,139.946471,82.386282,102.371693,223.012518,74.744194
8,Austria,2025-09-01,148.347559,81.830673,102.325115,208.005085,92.587667
9,Austria,2025-10-01,148.815134,82.227338,102.840521,208.005085,108.226452


In [ ]:
from utils.preprocessing import load_daily_data, load_monthly_data

daily_train, daily_val, daily_test = load_daily_data(use_existing=True)
monthly_train, monthly_val, monthly_test = load_monthly_data(use_existing=True)

,unique_id,ds,y
0,Austria,2015-01-01,22.34
1,Austria,2015-01-02,22.34
2,Austria,2015-01-03,22.34
3,Austria,2015-01-04,22.34
4,Austria,2015-01-05,36.18
...,...,...,...
3648,Austria,2024-12-27,121.79
3649,Austria,2024-12-28,122.12
3650,Austria,2024-12-29,110.49
3651,Austria,2024-12-30,118.13


# DEEP LEARNING


In [88]:
def createLag_lags(df):
    df = df.copy()
    df = mergeHolidays_daily(df)

    # 6 useful lag features
    grouped = df.groupby('unique_id')['y']

    df['lag1'] = grouped.transform(lambda x: x.shift(1)).fillna(0)         # yesterday
    df['lag7'] = grouped.transform(lambda x: x.shift(7)).fillna(0)         # 1 week ago
    df['lag28'] = grouped.transform(lambda x: x.shift(28)).fillna(0)         # 4 weeks ago
    df['lag365'] = grouped.transform(lambda x: x.shift(365)).fillna(0)         # 1 year ago
    df['rolling_mean_7'] = grouped.transform(lambda x: x.shift(1).rolling(7).mean()).fillna(0)   # weekly trend
    df['rolling_mean_30'] = grouped.transform(lambda x: x.shift(1).rolling(30).mean()).fillna(0)  # monthly trend

    return df


def createLag_lags_monthly(df, df_daily):
    df = df.copy()
    df = mergeHolidays_monthly(df, df_daily)

    # 6 useful lag features
    grouped = df.groupby('unique_id')['y']

    df['lag1'] = grouped.transform(lambda x: x.shift(1)).fillna(0)          # last month
    df['lag3'] = grouped.transform(lambda x: x.shift(3)).fillna(0)          # last quarter
    df['lag6'] = grouped.transform(lambda x: x.shift(6)).fillna(0)          # half-year
    df['lag12'] = grouped.transform(lambda x: x.shift(12)).fillna(0)        # last year

    df['rolling_mean_3'] = grouped.transform(lambda x: x.shift(1).rolling(3).mean()).fillna(0)
    # quarterly trend

    df['rolling_mean_12'] = grouped.transform(lambda x: x.shift(1).rolling(12).mean()).fillna(0)
    # yearly trend

    return df

## dl daily

In [ ]:
## Deep Learning Models:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, DeepAR, NLinear, KAN, NHITS, RNN, LSTM
from neuralforecast.losses.pytorch import MAE

h_val = 365
input_val = 365*4


models_dl = [
    NHITS(
        h=h_val,
        input_size=input_val,
        n_freq_downsample=[2, 1, 1],
        scaler_type='robust',
        max_steps=200,
        inference_windows_batch_size=1,
        learning_rate=1e-3,
    ),
    KAN(
        h=h_val,
        input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        learning_rate=1e-3,
        max_steps=500,
    ),
    RNN(
        h=h_val,
        input_size=input_val,
        inference_input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        max_steps=200,
    ),
    LSTM(
        input_size=input_val,
        h=h_val,
        max_steps=500,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
    ),
]


dl_forecast = NeuralForecast(models=models_dl, freq='D', )

dl_forecast.fit(df=daily_train)
fc_val = dl_forecast.predict(h_val)
fc_val

Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1


TypeError: NeuralForecast.__init__() got an unexpected keyword argument 'logger'

### daily with lag

In [ ]:


h_val = 365
input_val = 365*9
lags_columns = ['is_holiday', 'lag1', 'lag7', 'lag28', 'lag365', 'rolling_mean_7', 'rolling_mean_30']

train_lag = createLag_lags(daily_train)
val_lag = createLag_lags(daily_val)

models_dl_lags = [
    NHITS(
        h=h_val,
        input_size=input_val,
        n_freq_downsample=[2, 1, 1],
        scaler_type='robust',
        max_steps=200,
        inference_windows_batch_size=1,
        learning_rate=1e-3,
        hist_exog_list=lags_columns
    ),
    
    KAN(
        h=h_val,
        input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        learning_rate=1e-3,
        max_steps=500,
        hist_exog_list=lags_columns
    ),
    RNN(
        h=h_val,
        input_size=input_val,
        inference_input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        max_steps=200,
        hist_exog_list=lags_columns
    ),
    LSTM(
        input_size=input_val,
        h=h_val,
        max_steps=500,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        hist_exog_list=lags_columns
    ),
]


dl_forecast_lags = NeuralForecast(models=models_dl_lags, freq='D')
dl_forecast_lags.fit(df=train_lag)
fc_val_lags = dl_forecast_lags.predict(h_val)

C:\Users\loren\AppData\Local\Temp\ipykernel_24664\618623678.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['ds'].isin(aut_holidays.keys()).astype('int8')
C:\Users\loren\AppData\Local\Temp\ipykernel_24664\618623678.py:5: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['ds'].isin(aut_holidays.keys()).astype('int8')
Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name         | Type          | Params | Mode 
----------------------------------------

c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 11:   0%|          | 0/1 [00:00<?, ?it/s, v_num=18, train_loss_step=2.090, train_loss_epoch=2.090]        


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## monthly

In [ ]:
## Deep Learning Models:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, DeepAR, NLinear, KAN, NHITS, RNN, LSTM
from neuralforecast.losses.pytorch import MAE

h_val = 365
input_val = 365*4


models_dl = [
    NHITS(
        h=h_val,
        input_size=input_val,
        n_freq_downsample=[2, 1, 1],
        scaler_type='robust',
        max_steps=200,
        inference_windows_batch_size=1,
        learning_rate=1e-3,
    ),
    KAN(
        h=h_val,
        input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        learning_rate=1e-3,
        max_steps=500,
    ),
    RNN(
        h=h_val,
        input_size=input_val,
        inference_input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        max_steps=200,
    ),
    LSTM(
        input_size=input_val,
        h=h_val,
        max_steps=500,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
    ),
]


dl_forecast = NeuralForecast(models=models_dl, freq='MS')

dl_forecast.fit(df=monthly_train)
fc_val = dl_forecast.predict(h_val)
fc_val

### monthly with lag

In [90]:
h_val = 12
input_val = 12*8
lags_columns = ['count_holiday', 'lag1', 'lag3', 'lag6', 'lag12', 'rolling_mean_3', 'rolling_mean_12']
    
train_lag = createLag_lags_monthly(monthly_train, daily_train)
val_lag = createLag_lags_monthly(monthly_val, daily_val)

models_dl_lags = [
    NHITS(
        h=h_val,
        input_size=input_val,
        n_freq_downsample=[2, 1, 1],
        scaler_type='robust',
        max_steps=200,
        inference_windows_batch_size=1,
        learning_rate=1e-3,
        hist_exog_list=lags_columns
    ),
    
    KAN(
        h=h_val,
        input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        learning_rate=1e-3,
        max_steps=500,
        hist_exog_list=lags_columns
    ),
    RNN(
        h=h_val,
        input_size=input_val,
        inference_input_size=input_val,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        max_steps=200,
        hist_exog_list=lags_columns
    ),
    LSTM(
        input_size=input_val,
        h=h_val,
        max_steps=500,
        loss=MAE(),
        scaler_type='robust',
        encoder_n_layers=2,
        encoder_hidden_size=128,
        decoder_hidden_size=128,
        decoder_layers=2,
        hist_exog_list=lags_columns
    ),
]


dl_forecast_lags = NeuralForecast(models=models_dl_lags, freq='MS')
dl_forecast_lags.fit(df=train_lag)
fc_val_lags = dl_forecast_lags.predict(h_val)

Seed set to 1
Seed set to 1
Seed set to 1
Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 3.3 M  | train
-------------------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.260    Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 14.43it/s, v_num=20, train_loss_step=0.0392, train_loss_epoch=0.0392]

`Trainer.fit` stopped: `max_steps=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00, 14.02it/s, v_num=20, train_loss_step=0.0392, train_loss_epoch=0.0392]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | layers       | ModuleList    | 4.0 M  | train
-------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.974    Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode



Epoch 33:   0%|          | 0/1 [00:00<?, ?it/s, v_num=21, train_loss_step=0.261, train_loss_epoch=0.261]        


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\loren\anaconda3\envs\AIS3_TIS\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
